ИУ5-63Б Лупарев РК2 Вар. 14

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import time
import math


file_path = 'states_all.csv'
df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1715 entries, 0 to 1714
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1715 non-null   object 
 1   STATE                         1715 non-null   object 
 2   YEAR                          1715 non-null   int64  
 3   ENROLL                        1224 non-null   float64
 4   TOTAL_REVENUE                 1275 non-null   float64
 5   FEDERAL_REVENUE               1275 non-null   float64
 6   STATE_REVENUE                 1275 non-null   float64
 7   LOCAL_REVENUE                 1275 non-null   float64
 8   TOTAL_EXPENDITURE             1275 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1275 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1275 non-null   float64
 11  OTHER_EXPENDITURE             1224 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1275 non-null   float64
 13  GRA

In [17]:
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,NaN,NaN,731634.0,208.0,252.0,207.0,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,NaN,NaN,122487.0,NaN,NaN,NaN,NaN
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,NaN,NaN,673477.0,215.0,265.0,209.0,NaN
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,NaN,NaN,441490.0,210.0,256.0,211.0,NaN
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,NaN,NaN,5254844.0,208.0,261.0,202.0,NaN


In [19]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
YEAR,1715.0,2.002075e+03,9.568621e+00,1986.0,1994.00,2002.0,2010.00,2019.0
ENROLL,1224.0,9.175416e+05,1.066514e+06,43866.0,264514.50,649933.5,1010532.00,6307022.0
TOTAL_REVENUE,1275.0,9.102045e+06,1.175962e+07,465650.0,2189503.50,5085826.0,10845163.00,89217262.0
FEDERAL_REVENUE,1275.0,7.677799e+05,1.146992e+06,31020.0,189957.50,403548.0,827932.00,9990221.0
STATE_REVENUE,1275.0,4.223743e+06,5.549735e+06,0.0,1165776.00,2537754.0,5055547.50,50904567.0
LOCAL_REVENUE,1275.0,4.110522e+06,5.489562e+06,22093.0,715121.00,2058996.0,4755293.00,36105265.0
TOTAL_EXPENDITURE,1275.0,9.206242e+06,1.199279e+07,481665.0,2170404.00,5242672.0,10744201.50,85320133.0
INSTRUCTION_EXPENDITURE,1275.0,4.768010e+06,6.300569e+06,265549.0,1171336.50,2658253.0,5561959.00,43964520.0
SUPPORT_SERVICES_EXPENDITURE,1275.0,2.682587e+06,3.357214e+06,139963.0,638076.00,1525471.0,3222924.50,26058021.0
OTHER_EXPENDITURE,1224.0,4.299509e+05,5.347893e+05,11541.0,103449.25,271704.0,517222.25,3995951.0


In [20]:
print(df.isnull().sum())
print(f"\nDataset Shape: {df.shape}")

PRIMARY_KEY                        0
STATE                              0
YEAR                               0
ENROLL                           491
TOTAL_REVENUE                    440
FEDERAL_REVENUE                  440
STATE_REVENUE                    440
LOCAL_REVENUE                    440
TOTAL_EXPENDITURE                440
INSTRUCTION_EXPENDITURE          440
SUPPORT_SERVICES_EXPENDITURE     440
OTHER_EXPENDITURE                491
CAPITAL_OUTLAY_EXPENDITURE       440
GRADES_PK_G                      173
GRADES_KG_G                       83
GRADES_4_G                        83
GRADES_8_G                        83
GRADES_12_G                       83
GRADES_1_8_G                     695
GRADES_9_12_G                    644
GRADES_ALL_G                      83
AVG_MATH_4_SCORE                1150
AVG_MATH_8_SCORE                1113
AVG_READING_4_SCORE             1065
AVG_READING_8_SCORE             1153
dtype: int64

Dataset Shape: (1715, 25)


In [21]:
# Предобработка данных
target = 'AVG_MATH_8_SCORE'
score_columns = ['AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', 'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE']
features_to_drop = ['PRIMARY_KEY', 'GRADES_1_8_G', 'GRADES_9_12_G'] + [col for col in score_columns if col != target]

df_processed = df.drop(columns=features_to_drop)
df_processed.dropna(subset=[target], inplace=True)
X = df_processed.drop(columns=[target])
y = df_processed[target]
categorical_features = ['STATE']
numerical_features = X.select_dtypes(include=np.number).columns.tolist()


numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
# Используем ColumnTransformer из sklearn.compose
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ],
    remainder='passthrough'
)

In [22]:
# Разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=52)
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

ohe_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
all_feature_names = numerical_features + list(ohe_feature_names)
print("\nПропуски в обработанных данных для обучения:", np.isnan(X_train_processed).sum())
print("Пропуски в обработанных данных для теста:", np.isnan(X_test_processed).sum())


Пропуски в обработанных данных для обучения: 0
Пропуски в обработанных данных для теста: 0


In [23]:
# Дерево решений
dt_pipeline = Pipeline(steps=[('regressor', DecisionTreeRegressor(random_state=52))])
param_grid_dt = {
    'regressor__max_depth': [None, 10, 20, 30, 40, 50],
    'regressor__min_samples_split': [2, 5, 10, 20],
    'regressor__min_samples_leaf': [1, 3, 5, 10, 15],
    'regressor__max_features': ['sqrt', 'log2', 0.7, 1.0]
}

# Подбор параметров для дерева
grid_search_dt = GridSearchCV(dt_pipeline, param_grid_dt, cv=5,
                              scoring='neg_mean_squared_error',
                              n_jobs=-1, verbose=0)
grid_search_dt.fit(X_train_processed, y_train)

best_dt = grid_search_dt.best_estimator_
print(f"Лучшие параметры для дерева: {grid_search_dt.best_params_}")


# Случайный лес
rf_pipeline = Pipeline(steps=[('regressor', RandomForestRegressor(random_state=52, n_jobs=-1))])

param_grid_rf = {
    'regressor__n_estimators': [100, 200, 300, 400],
    'regressor__max_depth': [1, 2, 3, 5, 10],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 3, 5],
    'regressor__max_features': ['sqrt', 'log2', 0.7, 1.0]
}


# Подбор параметров для леса
grid_search_rf = GridSearchCV(rf_pipeline, param_grid_rf, cv=5,
                              scoring='neg_mean_squared_error',
                              n_jobs=-1, verbose=0)
grid_search_rf.fit(X_train_processed, y_train)

best_rf = grid_search_rf.best_estimator_
print(f"Лучшие параметры для леса: {grid_search_rf.best_params_}")

Лучшие параметры для дерева: {'regressor__max_depth': 30, 'regressor__max_features': 1.0, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 20}
Лучшие параметры для леса: {'regressor__max_depth': 10, 'regressor__max_features': 0.7, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 300}


In [24]:
# Функция оценки одной модели
def evaluate_model(name, model, X_test, y_test):
    predictions = model.predict(X_test)
    r2 = r2_score(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = math.sqrt(mse)
    return {'Model': name, 'R2': r2, 'MAE': mae, 'MSE': mse, 'RMSE': rmse}

# Оценка моделей
dt_metrics = evaluate_model("Decision Tree", best_dt, X_test_processed, y_test)
rf_metrics = evaluate_model("Random Forest", best_rf, X_test_processed, y_test)

# Сводная таблица результатов
metrics_summary = pd.DataFrame([dt_metrics, rf_metrics])
metrics_summary

,Model,R2,MAE,MSE,RMSE
0,Decision Tree,0.686761,3.566848,25.183982,5.018364
1,Random Forest,0.697819,3.810207,24.294904,4.928986
